In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# File paths and names
xlsx_path = "D:\\Armaghan_PhD_Dissertation\\database\\Revised data"
xlsx_file = "new data-3 - rev.xlsx"

# Get user inputs for sheet names
material1_sheet = input("Enter the first material sheet name: ")
material2_sheet = input("Enter the second material sheet name: ")

# Ensure only specified sheets are loaded
valid_sheets = ["Ti", "Al", "Cu"]
if material1_sheet not in valid_sheets or material2_sheet not in valid_sheets:
    print("Invalid sheet name(s). Please enter valid sheet names.")
    exit()

# Load material data
data = {}
for material_sheet in [material1_sheet, material2_sheet]:
    if material_sheet not in valid_sheets:
        print(f"Sheet '{material_sheet}' is not allowed. Please enter a valid sheet name.")
        exit()
    
    file_path = os.path.join(xlsx_path, xlsx_file)
    material_data = pd.read_excel(file_path, sheet_name=material_sheet)
    data[material_sheet] = material_data

# Take 50% of each material for each variable
combined_data = (data[material1_sheet].iloc[:, 1:] + data[material2_sheet].iloc[:, 1:]) / 2.0

# Extract temperature-related information
temperature_col = "T (K)"
temperature_data = data[material1_sheet][temperature_col]

# Combine sheet names
combined_material_name = material1_sheet + material2_sheet

# Extract properties for the specified temperature
temperature_input = float(input("Enter the temperature (in K): "))
closest_index = np.argmin(np.abs(temperature_data - temperature_input))
properties_at_temperature = combined_data.iloc[closest_index]

# Check if Band Gap result is zero
if properties_at_temperature['Band Gap (eV)'] == 0:
    band_gap_result = None
else:
    # Prepare data for training the Decision Tree model
    X = combined_data.values
    y = merged_data.iloc[:, 1:].values

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features and target
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    # Train the Decision Tree model
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train_scaled, y_train_scaled)

    # Predict properties at the specified temperature
    temperature_input_scaled = scaler_X.transform([properties_at_temperature.values[1:]])
    predicted_properties_scaled = model.predict(temperature_input_scaled.reshape(1, -1))
    predicted_properties = scaler_y.inverse_transform(predicted_properties_scaled)

    # Display predicted material properties for the combined material at the specified temperature
    band_gap_result = predicted_properties[0, 0]

# Display all material properties, excluding Band Gap if it's zero
print("\nMaterial Properties at {}K for {}:".format(temperature_input, combined_material_name))
for property_name, property_value in zip(data[material1_sheet].columns[1:], properties_at_temperature):
    if property_name != 'Band Gap (eV)' or band_gap_result is not None:
        print("{} at {}K: {:.6f}".format(combined_material_name, property_name, property_value))

# Display the Band Gap result if it's not zero
if band_gap_result is not None:
    print("Band Gap (eV) - Predicted: {:.6f}".format(band_gap_result))


Enter the first material sheet name: Ti
Enter the second material sheet name: Al
Enter the temperature (in K): 334.15

Material Properties at 334.15K for TiAl:
TiAl at T (K)K: 334.150000
TiAl at Young Module (GPa)K: 84.811949
TiAl at Density (g/cm3)K: 3.528597
TiAl at Energy Absorption (KJ/m3)K: 5777.869528
TiAl at Thermal Conductivity (W/m.K)K: 85.860227
TiAl at Specific Heat (J/g.C)K: 0.731348
